In [1]:
using DataFrames, CSV, Query
using JuMP, Cbc

fn = "MOWOG1entries.csv"
c1_list=["AS", "BS", "CS", "DS", "ES", "FS", "GS", "HS","SS","SSR"]
_run_groups = 2
_min_to_bump = 4

#Distribute an integer over N rational parts
#@assert distribute_int(10,3)==[4,3,3]
function distribute_int(a::T,n::T) where {T<:Integer}
    (num,den) = divrem(a,n)
    [ifelse(i<=den,num+1,num) for i=1:n]
end

distribute_int (generic function with 1 method)

In [2]:
df=CSV.read(fn); #Read the CSV to a DataFrame
rename!(df, Symbol("Modifier/PAX") => :Index)
rename!(df, Symbol("First Name")=> :FirstName)
rename!(df, Symbol("Last Name")=> :LastName)
delete!(df, Symbol("Segment Name"))
df[:IndexClass]=map(x-> ismissing(x[:Index]) ? x[:Class] |> String : x[:Index] |> String, eachrow(df))
df

,LastName,FirstName,Class,Group,Year,Make,Model,Index,IndexClass
1,Agosto,Andrew,GS,missing,2013,Ford,Focus ST,Z,Z
2,Anderson,Dwight,HS,missing,2015,Ford,Fiesta ST,Z,Z
3,Austin,Ivan,ES,missing,2004,Toyota,MR2,P,P
4,Bahl,Dave,BS,missing,1992,Chevrolet,Corvette,missing,BS
5,Barber,Adam,STS,missing,1988,Honda,CRX,P,P
6,Barclay,Aaron,N,missing,1996,Lexus,sc400,missing,N
7,Barclay,Chad,FS,missing,2010,Infiniti,G37xS,missing,FS
8,Baumbach,Drew,ASP,missing,2009,Porsche,Cayman S,missing,ASP
9,Beck,Robert,N,missing,2016,BMW,m235i,missing,N
10,Bedell,Jody,SSC,missing,2015,Scion,FR-S,Z,Z


In [3]:
#Count up the drivers per class
d=@from i in df begin
    @group i by i.IndexClass into g
    @select {Class=g.key, Count=length(g)}
    @collect Dict
end
#Divide up novice class
begin
    for i=1:_run_groups
        d["N$i"]=distribute_int(d["N"],_run_groups)[i]
    end
    delete!(d, "N")
end
d

Dict{String,Int64} with 26 entries:
  "Z"     => 30
  "STX"   => 2
  "ES"    => 5
  "SSP"   => 1
  "SSC"   => 2
  "P"     => 21
  "N1"    => 15
  "V"     => 3
  "N2"    => 14
  "SMF"   => 5
  "DS"    => 4
  "CS"    => 3
  "CAM-S" => 3
  "STU"   => 2
  "ASP"   => 1
  "SSM"   => 3
  "HS"    => 4
  "X"     => 2
  "STS"   => 2
  "STH"   => 1
  "GS"    => 1
  "DM"    => 2
  "BS"    => 7
  "EM"    => 2
  "FS"    => 1
  ⋮       => ⋮

In [4]:
# Create a model
N = length(d)
m = Model(solver=CbcSolver())

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is CbcMathProg

In [5]:
# Create our variables
@variable(m, x[classes=1:N, rungroup=1:_run_groups], Bin)

26×2 Array{JuMP.Variable,2}:
 x[1,1]   x[1,2] 
 x[2,1]   x[2,2] 
 x[3,1]   x[3,2] 
 x[4,1]   x[4,2] 
 x[5,1]   x[5,2] 
 x[6,1]   x[6,2] 
 x[7,1]   x[7,2] 
 x[8,1]   x[8,2] 
 x[9,1]   x[9,2] 
 x[10,1]  x[10,2]
 x[11,1]  x[11,2]
 x[12,1]  x[12,2]
 x[13,1]  x[13,2]
 x[14,1]  x[14,2]
 x[15,1]  x[15,2]
 x[16,1]  x[16,2]
 x[17,1]  x[17,2]
 x[18,1]  x[18,2]
 x[19,1]  x[19,2]
 x[20,1]  x[20,2]
 x[21,1]  x[21,2]
 x[22,1]  x[22,2]
 x[23,1]  x[23,2]
 x[24,1]  x[24,2]
 x[25,1]  x[25,2]
 x[26,1]  x[26,2]

In [6]:
#each class can only be in one run group
for class = 1:N
    @constraint(m, sum(x[class,:]) == 1)
end

In [7]:
#keep Combined classes together if necessary
let ind=[any(key.==c1_list) && val <= _min_to_bump for (key,val) in d] |> find
    "Combining $(join(collect(keys(d))[ind],',')) due to <= $_min_to_bump drivers" |> println
    for run_group=1:_run_groups
        @constraint(m, x[ind[1:end-1],run_group].==x[ind[2:end],run_group])
    end
end
let ind=[!any(key.==c1_list) && val <= _min_to_bump for (key,val) in d] |> find
    "Combining $(join(collect(keys(d))[ind],',')) due to <= $_min_to_bump drivers" |> println
    for run_group=1:_run_groups
        @constraint(m, x[ind[1:end-1],run_group].==x[ind[2:end],run_group])
    end
end

Combining DS,CS,HS,GS,FS due to <= 4 drivers
Combining STX,SSP,SSC,V,CAM-S,STU,ASP,SSM,X,STS,STH,DM,EM,CAM-T due to <= 4 drivers


In [8]:
#split novice classes, only one novice group per run group
let ind=map(x->x[1]=='N',keys(d)) |> find
    for run_group=1:_run_groups
        @constraint(m, sum(x[ind,run_group])==1)
    end
end

In [9]:
#split Pro & Z
let ind=[any(key.==["P","Z"]) && val <= _min_to_bump for (key,val) in d] |> find
    for run_group=1:_run_groups
        @constraint(m, sum(x[ind,run_group])<=1)
    end
end

In [10]:
#Sum the drivers per run group to a vector
@expression(m, drive_count[i=1:_run_groups], sum(x[:,i].*collect(values(d))))
#Constraint so that group1>=group>=group3, etc...
for i=1:_run_groups-1
    @constraint(m, drive_count[i]>=drive_count[i+1])
end
#Objective to minimize the size of Group1, which means Group2,3,etc. have to accumulate classes
@objective(m, Min, drive_count[1])

30 x[1,1] + 2 x[2,1] + 5 x[3,1] + x[4,1] + 2 x[5,1] + 21 x[6,1] + 15 x[7,1] + 3 x[8,1] + 14 x[9,1] + 5 x[10,1] + 4 x[11,1] + 3 x[12,1] + 3 x[13,1] + 2 x[14,1] + x[15,1] + 3 x[16,1] + 4 x[17,1] + 2 x[18,1] + 2 x[19,1] + x[20,1] + x[21,1] + 2 x[22,1] + 7 x[23,1] + 2 x[24,1] + x[25,1] + 3 x[26,1]

In [11]:
# Short description of the problem complexity
show(m)

Minimization problem with:
 * 65 linear constraints
 * 52 variables: 52 binary
Solver is CbcMathProg

In [12]:
status=solve(m)

:Optimal

In [16]:
let drivers=getvalue(drive_count)
    for i=1:length(drivers)
        println("Run group #$i has $(Integer(drivers[i])) drivers")
    end
end

Run group #1 has 70 drivers
Run group #2 has 69 drivers


In [14]:
#run group 1
collect(keys(d))[getvalue(x[:,1]).>0.0]

9-element Array{String,1}:
 "Z" 
 "ES"
 "N1"
 "DS"
 "CS"
 "HS"
 "GS"
 "BS"
 "FS"

In [15]:
# run group 2
collect(keys(d))[getvalue(x[:,2]).>0.0]

17-element Array{String,1}:
 "STX"  
 "SSP"  
 "SSC"  
 "P"    
 "V"    
 "N2"   
 "SMF"  
 "CAM-S"
 "STU"  
 "ASP"  
 "SSM"  
 "X"    
 "STS"  
 "STH"  
 "DM"   
 "EM"   
 "CAM-T"